In [18]:
import yfinance as yf
import numpy as np

class DCF:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)

    def calculate_fcf(self):
        cashflow = self.stock.cashflow
        if 'Free Cash Flow' in cashflow.index:
            fcf = cashflow.loc['Free Cash Flow']
        else:
            print("Free Cash Flow data is not available.")
            fcf = None
        return fcf
    
    def calculate_wacc(self):
        balance_sheet = self.stock.balance_sheet
        income_statement = self.stock.financials
        ebit=income_statement.loc['EBIT']
        interest_expense = income_statement.loc['Interest Expense']
        tax_rate = 0.125
        risk_free_rate = 0.02
        market_preimum = 0.05
        beta=1.2
        cost_of_equity=risk_free_rate+beta*market_preimum
        cost_of_debt=(interest_expense)*(1-tax_rate)
        wacc=(cost_of_equity+cost_of_debt)*(1+tax_rate)
        return wacc.dropna()
    
    def terminal_value(self):
        fcf = self.calculate_fcf()
        wacc = self.calculate_wacc()
        growth_rate=0.04
        terminal_value=fcf*(1+growth_rate)/(wacc-growth_rate)
        terminal_value=terminal_value.dropna()
        return terminal_value[::-1]

    def calculate_dcf(self):
        fcf = self.calculate_fcf()  # List of free cash flows
        wacc = self.calculate_wacc()  # List of weighted average cost of capital for each period
        terminal_value = self.terminal_value()  # Terminal value at the end of projection

        if len(fcf) != len(wacc) + 1:
            print("Error: The length of FCF should be one more than WACC for terminal value calculation.")
            return []

        dcf = []  # Initialize the list to store discounted cash flows
    
        # Calculate discounted FCF for each year except the terminal value
        for i in range(len(wacc)):
            discounted_fcf = fcf[i] / (1 + wacc[i]) ** (i + 1)
            dcf.append(discounted_fcf)
            # Debugging: Print each discounted cash flow
            print(f"Year {i+1}: FCF = {fcf[i]}, WACC = {wacc[i]}, Discounted FCF = {discounted_fcf}")

        # Add the terminal value, discounted using the last WACC
        terminal_value_discounted = (fcf[-1] + terminal_value) / (1 + wacc[-1]) ** len(fcf)
        dcf.append(terminal_value_discounted)

        # Debugging: Print the final discounted terminal value
        print("Discounted Terminal Value:", terminal_value_discounted)
        
        print("Final DCF:", dcf)  # Debugging: Print the final DCF list
        return dcf


        
    
    
    def get_outstanding_shares(self):
        try:
            shares = self.stock.info['sharesOutstanding']
        except KeyError as e:
            print(f"KeyError: {e}")
            shares = None
        return shares
    
    def calculate_projected_share_price(self, dcf_value):
        shares_outstanding = self.get_outstanding_shares()
        if dcf_value is not None and shares_outstanding is not None:
            projected_share_price = dcf_value / shares_outstanding
        else:
            print("Cannot calculate projected share price due to missing data.")
            projected_share_price = None
        return projected_share_price

# Example usage
dcf_model = DCF('AAPL')  # Replace 'AAPL' with the desired ticker symbol
dcf_value = dcf_model.calculate_dcf()


FCF: 2023-09-30     99584000000.0
2022-09-30    111443000000.0
2021-09-30     92953000000.0
2020-09-30     73365000000.0
2019-09-30               NaN
Name: Free Cash Flow, dtype: object
WACC: 2023-09-30    3871546875.09
2022-09-30    2885203125.09
2021-09-30    2603671875.09
2020-09-30    2828109375.09
Name: Interest Expense, dtype: object
Terminal Value: 2023-09-30    26.750899
2022-09-30    40.170731
2021-09-30    37.128765
2020-09-30    26.979013
Freq: -1YE-SEP, dtype: object
Year 1: FCF = 99584000000.0, WACC = 3871546875.09, Discounted FCF = 25.722018404326565
Year 2: FCF = 111443000000.0, WACC = 2885203125.09, Discounted FCF = 1.3387516110184354e-08
Year 3: FCF = 92953000000.0, WACC = 2603671875.09, Discounted FCF = 5.2662885936972126e-18
Year 4: FCF = 73365000000.0, WACC = 2828109375.09, Discounted FCF = 1.1468433888228896e-27
Discounted Terminal Value: 2023-09-30    NaN
2022-09-30    NaN
2021-09-30    NaN
2020-09-30    NaN
Freq: -1YE-SEP, dtype: object
Final DCF: [25.72201840432

/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_54822/624749636.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  discounted_fcf = fcf[i] / (1 + wacc[i]) ** (i + 1)
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_54822/624749636.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Year {i+1}: FCF = {fcf[i]}, WACC = {wacc[i]}, Discounted FCF = {discounted_fcf}")
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_54822/624749636.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (c

In [19]:
dcf_model.calculate_dcf()

FCF: 2023-09-30     99584000000.0
2022-09-30    111443000000.0
2021-09-30     92953000000.0
2020-09-30     73365000000.0
2019-09-30               NaN
Name: Free Cash Flow, dtype: object
WACC: 2023-09-30    3871546875.09
2022-09-30    2885203125.09
2021-09-30    2603671875.09
2020-09-30    2828109375.09
Name: Interest Expense, dtype: object
Terminal Value: 2023-09-30    26.750899
2022-09-30    40.170731
2021-09-30    37.128765
2020-09-30    26.979013
Freq: -1YE-SEP, dtype: object
Year 1: FCF = 99584000000.0, WACC = 3871546875.09, Discounted FCF = 25.722018404326565
Year 2: FCF = 111443000000.0, WACC = 2885203125.09, Discounted FCF = 1.3387516110184354e-08
Year 3: FCF = 92953000000.0, WACC = 2603671875.09, Discounted FCF = 5.2662885936972126e-18
Year 4: FCF = 73365000000.0, WACC = 2828109375.09, Discounted FCF = 1.1468433888228896e-27
Discounted Terminal Value: 2023-09-30    NaN
2022-09-30    NaN
2021-09-30    NaN
2020-09-30    NaN
Freq: -1YE-SEP, dtype: object
Final DCF: [25.72201840432

/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_54822/624749636.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  discounted_fcf = fcf[i] / (1 + wacc[i]) ** (i + 1)
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_54822/624749636.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Year {i+1}: FCF = {fcf[i]}, WACC = {wacc[i]}, Discounted FCF = {discounted_fcf}")
/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_54822/624749636.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (c

[25.722018404326565,
 1.3387516110184354e-08,
 5.2662885936972126e-18,
 1.1468433888228896e-27,
 2023-09-30    NaN
 2022-09-30    NaN
 2021-09-30    NaN
 2020-09-30    NaN
 Freq: -1YE-SEP, dtype: object]

In [16]:
print(len(dcf_model.calculate_fcf()))
print(len(dcf_model.calculate_wacc()))

5
4
